<a href="https://colab.research.google.com/github/softcomput/cp-colab/blob/main/cpp_to_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Getting started**

1. Install Google OR-Tools.
2. Create small data model or large data model.
3. Run car pooling problem to work.

In [ ]:
!pip install ortools

In [ ]:
import logging
import re

from ortools.linear_solver import pywraplp
import numpy as np
import itertools


class CPPToWork:
    """CPPToWork

    Attributes:
        data (dict):
            Vs (list): servers
            Vc (list): clients
            Q (list): capacities
            T (list): maximum ride times
            Ls (list): latest arrival time at workspace of servers
            Lc (list): latest arrival time at workspace of clients
            Ss (list): earliest departure time at home of servers
            Sc (list): earliest departure time at home of clients
            U (list): upper limited detour time of servers
            distance (2-d list): distance for each server and client
            duration (2-d list): duration for each server and client
            preference (2-d list): preference for each server and client
            risk (2-d list): risk for each server and client
    """

    def __init__(self, data):
        self.M = 1000000
        self.DT = self.__calculate_max_detour_time(data)
        self.d = self.__format_data(data)

    def __calculate_max_detour_time(self, data):
        DT = 0
        for k in range(len(data['Vs'])):
            DT += data['T'][k] - data['duration'][k][0]
        return max(data['U']) * DT

    def __format_data(self, data):
        """ __format_data

        Format data model.
        """
        distance = []
        for vs_d in data['distance'][0:len(data['Vs'])]:
            distance.append(vs_d)
            distance.extend(data['distance'][len(data['Vs']):])
        data['distance'] = distance

        duration = []
        for vs_d in data['duration'][0:len(data['Vs'])]:
            duration.append(vs_d)
            duration.extend(data['duration'][len(data['Vs']):])
        data['duration'] = duration

        distance_vc = []
        for vc in range(len(data['Vc'])):
            distance_vc.append(data['distance'][vc + 1][0])
        data['distance_vc'] = distance_vc

        l_vc = []
        for vs in range(len(data['Vs'])):
            l = []
            l.append(data['Ls'][vs])
            l.extend(data['Lc'])
            l_vc.append(l)
        data['L'] = l_vc

        s_vc = []
        for vs in range(len(data['Vs'])):
            s = []
            s.append(data['Ss'][vs])
            s.extend(data['Sc'])
            s_vc.append(s)
        data['S'] = s_vc

        if data.get('preference'):
            preference = []
            for vs_d in data['preference'][0:len(data['Vs'])]:
                preference.append(vs_d)
                preference.extend(data['preference'][len(data['Vs']):])
            data['preference'] = preference

        return data

    def __create_variables(self, solver):
        """ __create_variables

        Create variables.

        Returns:
            x (list): x^k_ij, k in Vs
            y (list): y_i, i in Vc
            u (list): u^k_i, Si <= u^k_i <= Li　- t_i0
            v (list): v^k, Sk + t_k0 <= v^k <= Lk　, k in Vs
            q (list): 0 <= q^k_i <= Q^k
        """
        x = []
        u = []
        q = []
        t_k0 = []
        for vs, k in enumerate(self.d['Vs']):
            ai = [k]
            ai.extend(self.d['Vc'])
            aj = [0]
            aj.extend(self.d['Vc'])
            for i in ai:
                x.append([
                    solver.IntVar(0, 1, 'x{}({},{})'.format(k, i, j))
                    for j in aj])

            t_k0_vs = self.d['duration'][(1 + len(self.d['Vc'])) * vs][0]
            t_i0 = [t_k0_vs]
            t_k0.append(t_k0_vs)
            for vs_i in range(len(ai[1:])):
                t_i0.append(self.d['duration'][
                    1 + (1 + len(self.d['Vc'])) * vs + vs_i][0])

            u.append([solver.IntVar(
                self.d['S'][vs][vs_i],
                self.d['L'][vs][vs_i] - t_i0[vs_i],
                'u{}({})'.format(k, i)) for vs_i, i in enumerate(ai)])

            q.append([solver.IntVar(
                1,
                self.d['Q'][vs],
                'q{}({})'.format(k, j))
                for j in aj])

        y = [solver.IntVar(0, 1, 'y({})'.format(i)) for i in self.d['Vc']]

        v = [solver.IntVar(
            self.d['Ss'][vs] + t_k0[vs],
            self.d['Ls'][vs],
            'v{}'.format(k)) for vs, k in enumerate(self.d['Vs'])]

        return x, y, u, v, q

    def __create_constraints(self, solver, x, y, u, v, q):
        """ __create_constraints

        Create constraints.
        """
        n = len(self.d['Vc']) + 1

        for vs in range(len(self.d['Vs'])):
            k = vs * n
            # Each employee leaves its house.
            solver.Add(solver.Sum(x[k]) == 1)

            # Each employee arrives at the workspace.
            solver.Add(solver.Sum([x[j][0] for j in range(k, k + n)]) == 1)

            # Continuity.
            for i in range(1, len(self.d['Vc']) + 1):
                solver.Add(
                    solver.Sum([x[j][i] for j in range(k, k + n)]) -
                    solver.Sum(x[k + i]) == 0)

            # Capacity.
            solver.Add(
                solver.Sum([x_k for i in range(n) for x_k in x[k + i]]) <=
                self.d['Q'][vs])

            # Maximum ride time.
            solver.Add(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    self.d['duration'][k:k + n], x[k:k + n])]) <=
                self.d['T'][vs])

            # Limited detour time.
            solver.Add(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    self.d['duration'][k:k + n], x[k:k + n])]) - self.d['duration'][k][0] <=
                self.DT / self.d['U'][vs])

            # Own arc is not passed.
            for i in range(1, len(self.d['Vc']) + 1):
                solver.Add(x[k + i][i] == 0)

            # Subtour elimination constraints by MTZ.
            for j in range(1, len(q[vs])):
                solver.Add(
                    q[vs][0] + 1 - self.d['Q'][vs] * (1 - x[k][j]) <= q[vs][j])
            for i in range(1, len(q[vs])):
                for j in range(1, len(q[vs])):
                    if str(q[vs][i]) != str(q[vs][j]):
                        solver.Add(
                            q[vs][i] + 1 -
                            self.d['Q'][vs] * (1 - x[k + i][j]) <=
                            q[vs][j])
            solver.Add(q[vs][0] == 1)

            # Departure time condition.
            for j in range(1, len(u[vs])):
                solver.Add(
                    u[vs][0] +
                    self.d['duration'][k][j] -
                    self.M * (1 - x[k][j]) <=
                    u[vs][j])
            for i in range(1, len(u[vs])):
                for j in range(1, len(u[vs])):
                    if str(u[vs][i]) != str(u[vs][j]):
                        solver.Add(
                            u[vs][i] +
                            self.d['duration'][k + i][j] -
                            self.M * (1 - x[k + i][j]) <=
                            u[vs][j])
            solver.Add(u[vs][0] == self.d['Ss'][vs])

            # Arrival time at workspace of server.
            for i in range(len(u[vs])):
                solver.Add(
                    v[vs] >=
                    u[vs][i] +
                    self.d['duration'][k + i][0] -
                    self.M * (1 - x[k + i][0]))

                solver.Add(
                    v[vs] <=
                    self.d['L'][vs][i] +
                    self.M * (1 - solver.Sum(x[k + i])))

        # Each client is either picked up by a server or is left unserviced.
        for vc in range(len(self.d['Vc'])):
            solver.Add(
                solver.Sum([x_k for vs in range(len(self.d['Vs']))
                           for x_k in x[vs * n + vc + 1]]) + y[vc] == 1)

    def __create_objective(self, solver, x, y):
        """ __create_objective

        Create objective.
        """
        if self.d.get('risk') and self.d.get('preference'):
            solver.Minimize(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    np.array(self.d['risk']) *
                    np.array(self.d['preference']) *
                    np.array(self.d['distance']), x)]) +
                solver.Sum([np.inner(np.array(self.d['distance_vc']) * 2, y)]))
        elif self.d.get('risk'):
            solver.Minimize(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    np.array(self.d['risk']) *
                    np.array(self.d['distance']), x)]) +
                solver.Sum([np.inner(np.array(self.d['distance_vc']) * 2, y)]))
        elif self.d.get('preference'):
            solver.Minimize(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    np.array(self.d['preference']) *
                    np.array(self.d['distance']), x)]) +
                solver.Sum([np.inner(np.array(self.d['distance_vc']) * 2, y)]))
        else:
            solver.Minimize(
                solver.Sum([np.inner(d, _x) for d, _x in zip(
                    self.d['distance'], x)]) +
                solver.Sum([np.inner(np.array(self.d['distance_vc']) * 2, y)]))

    def solve(self):
        """ solve

        Solve CPP.
        """
        solver = pywraplp.Solver(
            'cp', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

        x, y, u, v, q = self.__create_variables(solver)
        self.__create_constraints(solver, x, y, u, v, q)
        self.__create_objective(solver, x, y)

        status = solver.Solve()

        paths = {}
        clients_not_pickedup = []
        if status == pywraplp.Solver.OPTIMAL:
            n = len(self.d['Vc']) + 1
            root = []
            for i in range(len(self.d['Vs']) * n):
                for j in range(n):
                    if x[i][j].solution_value():
                        root.append(x[i][j].name())
                        r = re.findall(r'\w+', x[i][j].name())
                        r[0] = r[0].lstrip('x')
                        if not r[0] in paths:
                            paths[r[0]] = []
                        paths[r[0]].append((r[1], r[2]))
            paths = self.__format_paths(paths)
            for i in range(len(self.d['Vc'])):
                if y[i].solution_value():
                    r = re.findall(r'\w+', y[i].name())
                    clients_not_pickedup.append(r[1])
            arrival_times = []
            for i in range(len(self.d['Vs'])):
                arrival_times.append([
                    (u[i][j].name(), u[i][j].solution_value())
                    for j in range(len(u[i]))])
            logging.info(
                'Root %s, clients not picked up %s, \
                and arrival_times %s solved in %d variables, \
                %d constraints, %f objective value, %f milliseconds, \
                %d iterations, and %d branch-and-bound nodes.' % (
                    root, clients_not_pickedup, arrival_times,
                    solver.NumVariables(),
                    solver.NumConstraints(),
                    solver.Objective().Value(),
                    solver.wall_time(), solver.iterations(), solver.nodes()))
        else:
            logging.info('The problem does not have an optimal solution.')
        m, tdr, tis, sd = self.__evaluate(x, y)

        ala = self.__evaluate_accident_location_avoidance_rate(x, y)
        logging.info('ala: %s', ala)
        print('ala: %s', ala)

        if self.d.get('jaccard'):
            es = self.__evaluate_employee_satisfaction_rate(paths)
            logging.info('es: %s', es)
            print('es: %s', es)

        return {
            'feasible_paths': paths,
            '%m': m,
            '%tdr': tdr,
            '%tis': tis,
            'sd': sd,
            'clients_not_pickedup': clients_not_pickedup
        }

    def __format_paths(self, paths):
        for i in paths:
            clients = []
            next = paths[i][0][1]
            for _ in paths[i]:
                if not next == '0':
                    for j in paths[i]:
                        if next == j[0]:
                            clients.append(j[0])
                            next = j[1]
                            break
            paths[i] = clients

        return paths

    def __evaluate(self, x, y):
        total_distance = 0
        total_duration = 0
        original_distance = 0
        original_duration_vs = 0
        detour_times = []
        n = len(self.d['Vc']) + 1
        for i in range(len(self.d['Vs']) * n):
            if i % n == 0:
                detour_time = 0
            for j in range(n):
                if x[i][j].solution_value():
                    total_distance += self.d['distance'][i][j]
                    total_duration += self.d['duration'][i][j]
                    detour_time += self.d['duration'][i][j]
            if i % n == n - 1:
                detour_times.append(detour_time)
        for i in range(len(self.d['Vs'])):
            original_distance += self.d['distance'][i * n][0]
            original_duration_vs += self.d['duration'][i * n][0]
        not_pickup_clients = 0
        for i in range(len(self.d['Vc'])):
            if y[i].solution_value() == 1:
                not_pickup_clients += 1
                total_distance += self.d['distance'][i + 1][0]
            original_distance += self.d['distance'][i + 1][0]

        m = (len(self.d['Vc']) - not_pickup_clients) / len(self.d['Vc'])
        tdr = 1 - total_distance / original_distance
        tis = original_duration_vs / total_duration
        sd = np.sqrt(sum((detour_times - np.average(detour_times))**2 * self.d['U']) / sum(self.d['U']))
        return m, tdr, tis, sd

    def __evaluate_accident_location_avoidance_rate(self, x, y):
        if not self.d.get('accidentCount'):
            return 0
        total_accident_count = 0
        original_accident_count = 0
        n = len(self.d['Vc']) + 1
        for i in range(len(self.d['Vs']) * n):
            for j in range(n):
                if x[i][j].solution_value():
                    total_accident_count += self.d['accidentCount'][i][j]
        for i in range(len(self.d['Vs'])):
            original_accident_count += self.d['accidentCount'][i * n][0]
        for i in range(len(self.d['Vc'])):
            if y[i].solution_value() == 1:
                total_accident_count += self.d['accidentCount'][i + 1][0]
            original_accident_count += self.d['accidentCount'][i + 1][0]

        evasion_rate = 1 - total_accident_count / original_accident_count
        return evasion_rate

    def __evaluate_employee_satisfaction_rate(self, paths):
        n = len(self.d['Vs'])
        # 2-組合せを生成
        P = []
        C = []
        for i in paths:
            p = paths[i]
            p.insert(0, i)
            P.append(p)
            c = []
            combinations = list(itertools.combinations(p, 2))
            for pair in combinations:
                c.append(pair)
            C.append(c)
        jc = []
        for i, c in enumerate(C):
            # 運転者と同乗者の jaccard
            for j in range(len(P[i]) - 1):
                pair = c[j]
                vs = self.d['Vs'].index(pair[0])
                vc = self.d['Vc'].index(pair[1])
                jc.append(self.d['jaccard'][vs][vc + 1])
            # 同乗者同士の jaccard
            for j in range(len(P[i]) - 1, len(c)):
                pair = c[j]
                vc1 = self.d['Vc'].index(pair[0])
                vc2 = self.d['Vc'].index(pair[1])
                jc.append(self.d['jaccard'][len(self.d['Vs']) + vc1][vc2 + 1])
        return sum(jc) / len(jc)

data = create_data_model()
output = CPPToWork(data).solve()
print(output)

In [ ]:
def create_data_model():
    data = {}
    data['Vs'] = ['1', '2', '3', '4', '5', '6']
    data['Vc'] = ['7','8','9','10','11','12','13','14','15','16','17','18','19','20']
    data['Q'] = [5,5,5,5,5,5]
    data['T'] = [3600,3600,3600,3600,3600,3600]
    data['U'] = [20,20,20,20,20,20]
    data['Ls'] = [7200,7200,7200,7200,7200,7200]
    data['Lc'] = [7200,7200,7200,7200,7200,7200,7200,7200,7200,7200,7200,7200,7200,7200]
    data['Ss'] = [0,0,0,0,0,0]
    data['Sc'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    data['distance'] = [[6556,6850,3692,6225,8220,5232,7395,8073,5532,3692,21680,20400,10633,13076,20859],[8812,2675,4551,2316,3768,3236,9817,4137,2960,4551,24102,22822,13055,15498,23281],[9932,5296,5995,3077,4573,4559,10938,2716,5581,5995,29160,27880,14176,16619,28339],[4751,3190,1853,2841,4658,1415,5756,3758,3475,1853,20041,18761,8994,11437,19220],[16792,16773,16974,14233,16061,15154,11671,14223,17058,16974,5116,4181,10290,7804,2683],[15975,22778,21441,22300,26719,21003,10750,23216,23063,21441,785,2296,7860,5375,3181],[7734,0,3652,2239,4351,3159,8837,4720,809,3652,23122,21842,12075,14518,22301],[6006,3652,0,2717,4712,1724,6844,4663,3627,0,21129,19849,10082,12525,20308],[7230,2239,2717,0,2102,1576,8235,2159,2510,3013,22520,21240,11473,13916,21699],[12455,4351,4712,2102,0,5231,21587,4119,6164,6667,34435,33155,24824,27464,33614],[5959,3159,1724,1576,5231,0,7062,3153,3656,2034,21347,20067,10300,12743,20526],[2068,8837,6844,8235,21587,7062,0,9309,9156,7534,9521,14135,3238,5681,14594],[9005,4720,4663,2159,4119,3153,9309,0,4515,4968,23580,22300,12533,14976,17207],[8019,809,3627,2510,6164,3656,9156,4515,0,2662,23407,22127,12360,14803,22586],[6006,3652,0,3013,6667,2034,7534,4968,2662,0,21129,19849,10082,12525,20308],[15224,23122,21129,22520,34435,21347,9521,23580,23407,21129,0,1545,6625,4139,2430],[13928,21842,19849,21240,33155,20067,14135,22300,22127,19849,1545,0,7138,4652,1783],[5371,12075,10082,11473,24824,10300,3238,12533,12360,10082,6625,7138,0,2511,7262],[7754,14518,12525,13916,27464,12743,5681,14976,14803,12525,4139,4652,2511,0,5045],[14863,22301,20308,21699,33614,20526,14594,17207,22586,20308,2430,1783,7262,5045,0]]
    data['duration'] = [[722,915,461,895,1041,697,812,1115,723,461,1787,1629,1211,1491,1702],[1087,388,676,405,434,487,1210,745,392,676,2185,2027,1609,1889,2100],[1293,872,1020,594,619,783,1415,446,876,1020,2310,2152,1814,2094,2225],[504,580,370,543,587,293,626,648,584,370,1601,1443,1025,1305,1516],[1442,2086,2082,1776,1649,1823,1366,1773,2090,2082,842,745,1518,1178,466],[1399,2272,2062,2190,2170,1985,1342,2295,2276,2062,141,496,1109,770,594],[1014,0,579,433,514,524,1138,825,167,579,2113,1955,1537,1817,2028],[671,579,0,496,643,298,758,857,521,0,1733,1575,1157,1437,1648],[880,433,496,0,233,301,1002,474,414,538,1977,1819,1401,1681,1892],[969,514,643,233,0,567,1119,664,681,804,1952,1794,1518,1619,1867],[687,524,298,301,567,0,811,617,535,317,1786,1628,1210,1490,1701],[263,1138,758,1002,1119,811,0,1159,1140,926,1200,969,399,679,1042],[992,825,857,474,664,617,1159,0,832,839,2167,2009,1591,1871,2014],[1019,167,521,414,681,535,1140,832,0,396,2118,1960,1542,1822,2033],[671,579,0,538,804,317,926,839,396,0,1733,1575,1157,1437,1648],[1298,2113,1733,1977,1952,1786,1200,2167,2118,1733,0,395,1036,697,493],[1138,1955,1575,1819,1794,1628,969,2009,1960,1575,395,0,1165,826,470],[723,1537,1157,1401,1518,1210,399,1591,1542,1157,1036,1165,0,347,1029],[948,1817,1437,1681,1619,1490,679,1871,1822,1437,697,826,347,0,784],[1182,2028,1648,1892,1867,1701,1042,2014,2033,1648,493,470,1029,784,0]]
    data['risk'] = [[2.5,1,1,1.4166666666666667,1,1,1,1,1,1,1,1,1.5833333333333335,1.3333333333333333,1.4166666666666667],[2.5,0,1,1.4166666666666667,1,1.3333333333333333,1,1.3333333333333333,1,1,1,1,1.5833333333333335,1.3333333333333333,1],[2.5,1,0,1.5833333333333335,1,1,1,1,1,1,1,1,1.5833333333333335,1.3333333333333333,1],[2.5,1.4166666666666667,1.5833333333333335,0,1,1.5833333333333335,1.3333333333333333,1,1.3333333333333333,1.4166666666666667,1.3333333333333333,1,1.5833333333333335,1.3333333333333333,1.4166666666666667],[1,1,1,1,0,1.5833333333333335,1,1,1.3333333333333333,1.4166666666666667,1,1,1,1,1],[2.5,1.3333333333333333,1,1.5833333333333335,1.5833333333333335,0,1,1,1.6666666666666665,1,1.3333333333333333,2.5,1.5833333333333335,1.3333333333333333,1],[1,1,1,1.3333333333333333,1,1,0,1.5,1,1,1,1.6666666666666665,1,1.3333333333333333,1],[1.5,1.3333333333333333,1,1,1,1,1.5,0,1.3333333333333333,1.3333333333333333,1.3333333333333333,1,1.5833333333333335,1.3333333333333333,1.6666666666666665],[2.5,1,1,1.3333333333333333,1.3333333333333333,1.6666666666666665,1,1.3333333333333333,0,1,1,1,1.5833333333333335,1.3333333333333333,1],[2.5,1,1,1.4166666666666667,1.4166666666666667,1,1,1.3333333333333333,1,0,1,1,1.5833333333333335,1.3333333333333333,1],[1.3333333333333333,1,1,1.3333333333333333,1,1.3333333333333333,1,1.3333333333333333,1,1,0,1.9166666666666667,1,1.3333333333333333,1.4166666666666667],[2.5,1,1,1,1,2.5,1.6666666666666665,1,1,1,1.9166666666666667,0,1,1.3333333333333333,1.4166666666666667],[1.5833333333333335,1.5833333333333335,1.5833333333333335,1.5833333333333335,1,1.5833333333333335,1,1.5833333333333335,1.5833333333333335,1.5833333333333335,1,1,0,1,1.4166666666666667],[1.5833333333333335,1.3333333333333333,1.3333333333333333,1.3333333333333333,1,1.3333333333333333,1.3333333333333333,1.3333333333333333,1.3333333333333333,1.3333333333333333,1.3333333333333333,1.3333333333333333,1,0,1.4166666666666667],[1,1,1,1.4166666666666667,1,1,1,1.6666666666666665,1,1,1.4166666666666667,1.4166666666666667,1.4166666666666667,1.4166666666666667,0],[2.625,1.125,1.125,1.7916666666666665,1.125,1.4583333333333333,1.125,1.625,1.125,1.125,2.041666666666667,2.041666666666667,1.125,1.4583333333333333,1.125],[2.625,0,1.125,1.5416666666666667,1.125,1.4583333333333333,1.125,1.4583333333333333,1.125,1.125,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[2.625,1.125,0,1.7083333333333335,1.125,1.125,1.125,1.125,1.125,1.125,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[2.625,1.5416666666666667,1.7083333333333335,0,1.125,1.7083333333333335,1.4583333333333333,1.125,1.4583333333333333,1.5416666666666667,1.4583333333333333,1.125,1.7083333333333335,1.4583333333333333,1.5416666666666667],[1.125,1.125,1.125,1.125,0,1.7083333333333335,1.125,1.125,1.4583333333333333,1.5416666666666667,1.125,1.125,1.125,1.125,1.125],[2.625,1.4583333333333333,1.125,1.7083333333333335,1.7083333333333335,0,1.125,1.125,1.7916666666666665,1.125,1.4583333333333333,2.625,1.7083333333333335,1.4583333333333333,1.125],[1.125,1.125,1.125,1.4583333333333333,1.125,1.125,0,1.625,1.125,1.125,1.125,1.7916666666666665,1.125,1.4583333333333333,1.125],[1.625,1.4583333333333333,1.125,1.125,1.125,1.125,1.625,0,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.125,1.7083333333333335,1.4583333333333333,1.7916666666666665],[2.625,1.125,1.125,1.4583333333333333,1.4583333333333333,1.7916666666666665,1.125,1.4583333333333333,0,1.125,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[2.625,1.125,1.125,1.5416666666666667,1.5416666666666667,1.125,1.125,1.4583333333333333,1.125,0,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[1.4583333333333333,1.125,1.125,1.4583333333333333,1.125,1.4583333333333333,1.125,1.4583333333333333,1.125,1.125,0,2.041666666666667,1.125,1.4583333333333333,1.5416666666666667],[2.625,1.125,1.125,1.125,1.125,2.625,1.7916666666666665,1.125,1.125,1.125,2.041666666666667,0,1.125,1.4583333333333333,1.5416666666666667],[1.7083333333333335,1.7083333333333335,1.7083333333333335,1.7083333333333335,1.125,1.7083333333333335,1.125,1.7083333333333335,1.7083333333333335,1.7083333333333335,1.125,1.125,0,1.125,1.5416666666666667],[1.7083333333333335,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.125,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.125,0,1.5416666666666667],[1.125,1.125,1.125,1.5416666666666667,1.125,1.125,1.125,1.7916666666666665,1.125,1.125,1.5416666666666667,1.5416666666666667,1.5416666666666667,1.5416666666666667,0],[2.5,1.4166666666666667,1.3333333333333333,1.4166666666666667,1.4166666666666667,1.4166666666666667,1.5,1.5833333333333335,1.4166666666666667,1.3333333333333333,1,1,1.5833333333333335,1.4166666666666667,1],[2.5,0,1,1.4166666666666667,1,1.3333333333333333,1,1.3333333333333333,1,1,1,1,1.5833333333333335,1.3333333333333333,1],[2.5,1,0,1.5833333333333335,1,1,1,1,1,1,1,1,1.5833333333333335,1.3333333333333333,1],[2.5,1.4166666666666667,1.5833333333333335,0,1,1.5833333333333335,1.3333333333333333,1,1.3333333333333333,1.4166666666666667,1.3333333333333333,1,1.5833333333333335,1.3333333333333333,1.4166666666666667],[1,1,1,1,0,1.5833333333333335,1,1,1.3333333333333333,1.4166666666666667,1,1,1,1,1],[2.5,1.3333333333333333,1,1.5833333333333335,1.5833333333333335,0,1,1,1.6666666666666665,1,1.3333333333333333,2.5,1.5833333333333335,1.3333333333333333,1],[1,1,1,1.3333333333333333,1,1,0,1.5,1,1,1,1.6666666666666665,1,1.3333333333333333,1],[1.5,1.3333333333333333,1,1,1,1,1.5,0,1.3333333333333333,1.3333333333333333,1.3333333333333333,1,1.5833333333333335,1.3333333333333333,1.6666666666666665],[2.5,1,1,1.3333333333333333,1.3333333333333333,1.6666666666666665,1,1.3333333333333333,0,1,1,1,1.5833333333333335,1.3333333333333333,1],[2.5,1,1,1.4166666666666667,1.4166666666666667,1,1,1.3333333333333333,1,0,1,1,1.5833333333333335,1.3333333333333333,1],[1.3333333333333333,1,1,1.3333333333333333,1,1.3333333333333333,1,1.3333333333333333,1,1,0,1.9166666666666667,1,1.3333333333333333,1.4166666666666667],[2.5,1,1,1,1,2.5,1.6666666666666665,1,1,1,1.9166666666666667,0,1,1.3333333333333333,1.4166666666666667],[1.5833333333333335,1.5833333333333335,1.5833333333333335,1.5833333333333335,1,1.5833333333333335,1,1.5833333333333335,1.5833333333333335,1.5833333333333335,1,1,0,1,1.4166666666666667],[1.5833333333333335,1.3333333333333333,1.3333333333333333,1.3333333333333333,1,1.3333333333333333,1.3333333333333333,1.3333333333333333,1.3333333333333333,1.3333333333333333,1.3333333333333333,1.3333333333333333,1,0,1.4166666666666667],[1,1,1,1.4166666666666667,1,1,1,1.6666666666666665,1,1,1.4166666666666667,1.4166666666666667,1.4166666666666667,1.4166666666666667,0],[2.625,1.7916666666666665,1.125,1.4583333333333333,1.125,1.125,1.125,1.125,1.7916666666666665,1.125,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[2.625,0,1.125,1.5416666666666667,1.125,1.4583333333333333,1.125,1.4583333333333333,1.125,1.125,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[2.625,1.125,0,1.7083333333333335,1.125,1.125,1.125,1.125,1.125,1.125,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[2.625,1.5416666666666667,1.7083333333333335,0,1.125,1.7083333333333335,1.4583333333333333,1.125,1.4583333333333333,1.5416666666666667,1.4583333333333333,1.125,1.7083333333333335,1.4583333333333333,1.5416666666666667],[1.125,1.125,1.125,1.125,0,1.7083333333333335,1.125,1.125,1.4583333333333333,1.5416666666666667,1.125,1.125,1.125,1.125,1.125],[2.625,1.4583333333333333,1.125,1.7083333333333335,1.7083333333333335,0,1.125,1.125,1.7916666666666665,1.125,1.4583333333333333,2.625,1.7083333333333335,1.4583333333333333,1.125],[1.125,1.125,1.125,1.4583333333333333,1.125,1.125,0,1.625,1.125,1.125,1.125,1.7916666666666665,1.125,1.4583333333333333,1.125],[1.625,1.4583333333333333,1.125,1.125,1.125,1.125,1.625,0,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.125,1.7083333333333335,1.4583333333333333,1.7916666666666665],[2.625,1.125,1.125,1.4583333333333333,1.4583333333333333,1.7916666666666665,1.125,1.4583333333333333,0,1.125,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[2.625,1.125,1.125,1.5416666666666667,1.5416666666666667,1.125,1.125,1.4583333333333333,1.125,0,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[1.4583333333333333,1.125,1.125,1.4583333333333333,1.125,1.4583333333333333,1.125,1.4583333333333333,1.125,1.125,0,2.041666666666667,1.125,1.4583333333333333,1.5416666666666667],[2.625,1.125,1.125,1.125,1.125,2.625,1.7916666666666665,1.125,1.125,1.125,2.041666666666667,0,1.125,1.4583333333333333,1.5416666666666667],[1.7083333333333335,1.7083333333333335,1.7083333333333335,1.7083333333333335,1.125,1.7083333333333335,1.125,1.7083333333333335,1.7083333333333335,1.7083333333333335,1.125,1.125,0,1.125,1.5416666666666667],[1.7083333333333335,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.125,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.125,0,1.5416666666666667],[1.125,1.125,1.125,1.5416666666666667,1.125,1.125,1.125,1.7916666666666665,1.125,1.125,1.5416666666666667,1.5416666666666667,1.5416666666666667,1.5416666666666667,0],[1,1,1,1.5833333333333335,1,1.5833333333333335,1,1.5833333333333335,1,1,1.4166666666666667,1.9166666666666667,1,1.3333333333333333,1.4166666666666667],[2.5,0,1,1.4166666666666667,1,1.3333333333333333,1,1.3333333333333333,1,1,1,1,1.5833333333333335,1.3333333333333333,1],[2.5,1,0,1.5833333333333335,1,1,1,1,1,1,1,1,1.5833333333333335,1.3333333333333333,1],[2.5,1.4166666666666667,1.5833333333333335,0,1,1.5833333333333335,1.3333333333333333,1,1.3333333333333333,1.4166666666666667,1.3333333333333333,1,1.5833333333333335,1.3333333333333333,1.4166666666666667],[1,1,1,1,0,1.5833333333333335,1,1,1.3333333333333333,1.4166666666666667,1,1,1,1,1],[2.5,1.3333333333333333,1,1.5833333333333335,1.5833333333333335,0,1,1,1.6666666666666665,1,1.3333333333333333,2.5,1.5833333333333335,1.3333333333333333,1],[1,1,1,1.3333333333333333,1,1,0,1.5,1,1,1,1.6666666666666665,1,1.3333333333333333,1],[1.5,1.3333333333333333,1,1,1,1,1.5,0,1.3333333333333333,1.3333333333333333,1.3333333333333333,1,1.5833333333333335,1.3333333333333333,1.6666666666666665],[2.5,1,1,1.3333333333333333,1.3333333333333333,1.6666666666666665,1,1.3333333333333333,0,1,1,1,1.5833333333333335,1.3333333333333333,1],[2.5,1,1,1.4166666666666667,1.4166666666666667,1,1,1.3333333333333333,1,0,1,1,1.5833333333333335,1.3333333333333333,1],[1.3333333333333333,1,1,1.3333333333333333,1,1.3333333333333333,1,1.3333333333333333,1,1,0,1.9166666666666667,1,1.3333333333333333,1.4166666666666667],[2.5,1,1,1,1,2.5,1.6666666666666665,1,1,1,1.9166666666666667,0,1,1.3333333333333333,1.4166666666666667],[1.5833333333333335,1.5833333333333335,1.5833333333333335,1.5833333333333335,1,1.5833333333333335,1,1.5833333333333335,1.5833333333333335,1.5833333333333335,1,1,0,1,1.4166666666666667],[1.5833333333333335,1.3333333333333333,1.3333333333333333,1.3333333333333333,1,1.3333333333333333,1.3333333333333333,1.3333333333333333,1.3333333333333333,1.3333333333333333,1.3333333333333333,1.3333333333333333,1,0,1.4166666666666667],[1,1,1,1.4166666666666667,1,1,1,1.6666666666666665,1,1,1.4166666666666667,1.4166666666666667,1.4166666666666667,1.4166666666666667,0],[1.125,1.125,1.4583333333333333,1.125,1.4583333333333333,1.4583333333333333,1.125,1.4583333333333333,1.125,1.4583333333333333,1.125,2.041666666666667,1.125,1.4583333333333333,1.5416666666666667],[2.625,0,1.125,1.5416666666666667,1.125,1.4583333333333333,1.125,1.4583333333333333,1.125,1.125,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[2.625,1.125,0,1.7083333333333335,1.125,1.125,1.125,1.125,1.125,1.125,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[2.625,1.5416666666666667,1.7083333333333335,0,1.125,1.7083333333333335,1.4583333333333333,1.125,1.4583333333333333,1.5416666666666667,1.4583333333333333,1.125,1.7083333333333335,1.4583333333333333,1.5416666666666667],[1.125,1.125,1.125,1.125,0,1.7083333333333335,1.125,1.125,1.4583333333333333,1.5416666666666667,1.125,1.125,1.125,1.125,1.125],[2.625,1.4583333333333333,1.125,1.7083333333333335,1.7083333333333335,0,1.125,1.125,1.7916666666666665,1.125,1.4583333333333333,2.625,1.7083333333333335,1.4583333333333333,1.125],[1.125,1.125,1.125,1.4583333333333333,1.125,1.125,0,1.625,1.125,1.125,1.125,1.7916666666666665,1.125,1.4583333333333333,1.125],[1.625,1.4583333333333333,1.125,1.125,1.125,1.125,1.625,0,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.125,1.7083333333333335,1.4583333333333333,1.7916666666666665],[2.625,1.125,1.125,1.4583333333333333,1.4583333333333333,1.7916666666666665,1.125,1.4583333333333333,0,1.125,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[2.625,1.125,1.125,1.5416666666666667,1.5416666666666667,1.125,1.125,1.4583333333333333,1.125,0,1.125,1.125,1.7083333333333335,1.4583333333333333,1.125],[1.4583333333333333,1.125,1.125,1.4583333333333333,1.125,1.4583333333333333,1.125,1.4583333333333333,1.125,1.125,0,2.041666666666667,1.125,1.4583333333333333,1.5416666666666667],[2.625,1.125,1.125,1.125,1.125,2.625,1.7916666666666665,1.125,1.125,1.125,2.041666666666667,0,1.125,1.4583333333333333,1.5416666666666667],[1.7083333333333335,1.7083333333333335,1.7083333333333335,1.7083333333333335,1.125,1.7083333333333335,1.125,1.7083333333333335,1.7083333333333335,1.7083333333333335,1.125,1.125,0,1.125,1.5416666666666667],[1.7083333333333335,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.125,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.4583333333333333,1.125,0,1.5416666666666667],[1.125,1.125,1.125,1.5416666666666667,1.125,1.125,1.125,1.7916666666666665,1.125,1.125,1.5416666666666667,1.5416666666666667,1.5416666666666667,1.5416666666666667,0]]
    data['accidentCount'] = [[18,0,0,5,0,0,0,0,0,0,0,0,7,4,5],[18,0,0,5,0,4,0,4,0,0,0,0,7,4,0],[18,0,0,7,0,0,0,0,0,0,0,0,7,4,0],[18,5,7,0,0,7,4,0,4,5,4,0,7,4,5],[0,0,0,0,0,7,0,0,4,5,0,0,0,0,0],[18,4,0,7,7,0,0,0,8,0,4,18,7,4,0],[0,0,0,4,0,0,0,6,0,0,0,8,0,4,0],[6,4,0,0,0,0,6,0,4,4,4,0,7,4,8],[18,0,0,4,4,8,0,4,0,0,0,0,7,4,0],[18,0,0,5,5,0,0,4,0,0,0,0,7,4,0],[4,0,0,4,0,4,0,4,0,0,0,11,0,4,5],[18,0,0,0,0,18,8,0,0,0,11,0,0,4,5],[7,7,7,7,0,7,0,7,7,7,0,0,0,0,5],[7,4,4,4,0,4,4,4,4,4,4,4,0,0,5],[0,0,0,5,0,0,0,8,0,0,5,5,5,5,0],[18,0,0,8,0,4,0,6,0,0,11,11,0,4,0],[18,0,0,5,0,4,0,4,0,0,0,0,7,4,0],[18,0,0,7,0,0,0,0,0,0,0,0,7,4,0],[18,5,7,0,0,7,4,0,4,5,4,0,7,4,5],[0,0,0,0,0,7,0,0,4,5,0,0,0,0,0],[18,4,0,7,7,0,0,0,8,0,4,18,7,4,0],[0,0,0,4,0,0,0,6,0,0,0,8,0,4,0],[6,4,0,0,0,0,6,0,4,4,4,0,7,4,8],[18,0,0,4,4,8,0,4,0,0,0,0,7,4,0],[18,0,0,5,5,0,0,4,0,0,0,0,7,4,0],[4,0,0,4,0,4,0,4,0,0,0,11,0,4,5],[18,0,0,0,0,18,8,0,0,0,11,0,0,4,5],[7,7,7,7,0,7,0,7,7,7,0,0,0,0,5],[7,4,4,4,0,4,4,4,4,4,4,4,0,0,5],[0,0,0,5,0,0,0,8,0,0,5,5,5,5,0],[18,5,4,5,5,5,6,7,5,4,0,0,7,5,0],[18,0,0,5,0,4,0,4,0,0,0,0,7,4,0],[18,0,0,7,0,0,0,0,0,0,0,0,7,4,0],[18,5,7,0,0,7,4,0,4,5,4,0,7,4,5],[0,0,0,0,0,7,0,0,4,5,0,0,0,0,0],[18,4,0,7,7,0,0,0,8,0,4,18,7,4,0],[0,0,0,4,0,0,0,6,0,0,0,8,0,4,0],[6,4,0,0,0,0,6,0,4,4,4,0,7,4,8],[18,0,0,4,4,8,0,4,0,0,0,0,7,4,0],[18,0,0,5,5,0,0,4,0,0,0,0,7,4,0],[4,0,0,4,0,4,0,4,0,0,0,11,0,4,5],[18,0,0,0,0,18,8,0,0,0,11,0,0,4,5],[7,7,7,7,0,7,0,7,7,7,0,0,0,0,5],[7,4,4,4,0,4,4,4,4,4,4,4,0,0,5],[0,0,0,5,0,0,0,8,0,0,5,5,5,5,0],[18,8,0,4,0,0,0,0,8,0,0,0,7,4,0],[18,0,0,5,0,4,0,4,0,0,0,0,7,4,0],[18,0,0,7,0,0,0,0,0,0,0,0,7,4,0],[18,5,7,0,0,7,4,0,4,5,4,0,7,4,5],[0,0,0,0,0,7,0,0,4,5,0,0,0,0,0],[18,4,0,7,7,0,0,0,8,0,4,18,7,4,0],[0,0,0,4,0,0,0,6,0,0,0,8,0,4,0],[6,4,0,0,0,0,6,0,4,4,4,0,7,4,8],[18,0,0,4,4,8,0,4,0,0,0,0,7,4,0],[18,0,0,5,5,0,0,4,0,0,0,0,7,4,0],[4,0,0,4,0,4,0,4,0,0,0,11,0,4,5],[18,0,0,0,0,18,8,0,0,0,11,0,0,4,5],[7,7,7,7,0,7,0,7,7,7,0,0,0,0,5],[7,4,4,4,0,4,4,4,4,4,4,4,0,0,5],[0,0,0,5,0,0,0,8,0,0,5,5,5,5,0],[0,0,0,7,0,7,0,7,0,0,5,11,0,4,5],[18,0,0,5,0,4,0,4,0,0,0,0,7,4,0],[18,0,0,7,0,0,0,0,0,0,0,0,7,4,0],[18,5,7,0,0,7,4,0,4,5,4,0,7,4,5],[0,0,0,0,0,7,0,0,4,5,0,0,0,0,0],[18,4,0,7,7,0,0,0,8,0,4,18,7,4,0],[0,0,0,4,0,0,0,6,0,0,0,8,0,4,0],[6,4,0,0,0,0,6,0,4,4,4,0,7,4,8],[18,0,0,4,4,8,0,4,0,0,0,0,7,4,0],[18,0,0,5,5,0,0,4,0,0,0,0,7,4,0],[4,0,0,4,0,4,0,4,0,0,0,11,0,4,5],[18,0,0,0,0,18,8,0,0,0,11,0,0,4,5],[7,7,7,7,0,7,0,7,7,7,0,0,0,0,5],[7,4,4,4,0,4,4,4,4,4,4,4,0,0,5],[0,0,0,5,0,0,0,8,0,0,5,5,5,5,0],[0,0,4,0,4,4,0,4,0,4,0,11,0,4,5],[18,0,0,5,0,4,0,4,0,0,0,0,7,4,0],[18,0,0,7,0,0,0,0,0,0,0,0,7,4,0],[18,5,7,0,0,7,4,0,4,5,4,0,7,4,5],[0,0,0,0,0,7,0,0,4,5,0,0,0,0,0],[18,4,0,7,7,0,0,0,8,0,4,18,7,4,0],[0,0,0,4,0,0,0,6,0,0,0,8,0,4,0],[6,4,0,0,0,0,6,0,4,4,4,0,7,4,8],[18,0,0,4,4,8,0,4,0,0,0,0,7,4,0],[18,0,0,5,5,0,0,4,0,0,0,0,7,4,0],[4,0,0,4,0,4,0,4,0,0,0,11,0,4,5],[18,0,0,0,0,18,8,0,0,0,11,0,0,4,5],[7,7,7,7,0,7,0,7,7,7,0,0,0,0,5],[7,4,4,4,0,4,4,4,4,4,4,4,0,0,5],[0,0,0,5,0,0,0,8,0,0,5,5,5,5,0]]
    data['preference'] = [[1,0.33333333333333337,0.33333333333333337,0,0,0,0.6666666666666667,0.33333333333333337,0.33333333333333337,0,0,0.5,0,0.33333333333333337,0],[1,0,0.33333333333333337,0,0,0.33333333333333337,0,0.33333333333333337,0.6666666666666667,0,0,0.5,0,0,0],[1,0.25,0,0,0,0.25,0.33333333333333337,0.33333333333333337,0,0,0,0,0,0.25,0],[1,0,0,0,0,0.5,0,0.5,0.5,0,0,0.5,0,0,0],[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0.25,0.25,0,0,0.25,0.33333333333333337,0,0,0,0,0.5,0,0.25,0],[1,1,0.25,0,0,0.25,0,0.33333333333333337,0.33333333333333337,0,0,0.5,0,0.25,0],[1,0.25,1,0,0,0.25,0.33333333333333337,0.33333333333333337,0,0,0,0,0,0.25,0],[1,0,0,1,0,0,0,0,0,0,0,0,0,0,0],[1,0,0,0,1,0,0,0,0,0,0,0,0,0,0],[1,0.25,0.25,0,0,1,0.33333333333333337,0.33333333333333337,0.33333333333333337,0,0,0,0,0.25,0],[1,0,0.33333333333333337,0,0,0.33333333333333337,1,0.33333333333333337,0.6666666666666667,0,0,0.5,0,0,0],[1,0.33333333333333337,0.33333333333333337,0,0,0.33333333333333337,0.33333333333333337,1,0.33333333333333337,0,0,0.5,0,0,0],[1,0.33333333333333337,0,0,0,0.33333333333333337,0.6666666666666667,0.33333333333333337,1,0,0,0.5,0,0.33333333333333337,0],[1,0,0,0,0,0,0,0,0,1,0,0,0,0,0],[1,0,0,0,0,0,0,0,0,0,1,0,0,0,0],[1,0.5,0,0,0,0,0.5,0.5,0.5,0,0,1,0,0,0],[1,0,0,0,0,0,0,0,0,0,0,0,1,0,0],[1,0.25,0.25,0,0,0.25,0,0,0.33333333333333337,0,0,0,0,1,0],[1,0,0,0,0,0,0,0,0,0,0,0,0,0,1]]
    data['jaccard'] = [[0,0.4,0.4,0.6,0.6,0.75,0.2,0.5,0.5,0.6,0.6,0.25,0.6,0.4,0.6],[0,0.75,0.4,0.6,0.6,0.4,1,0.5,0.2,0.6,0.6,0.25,0.6,0.75,0.6],[0,0.6,1,0.8,0.8,0.6,0.4,0.4,0.75,0.8,0.8,0.5,0.8,0.6,0.8],[0,0.5,0.5,0.4,0.4,0.2,0.6666666666666666,0.25,0.25,0.4,0.4,0.3333333333333333,0.4,0.5,0.4],[0,0.8,0.8,1,1,0.8,0.6,0.6,0.6,1,1,0.4,1,0.8,1],[0,0.6,0.6,0.8,0.8,0.6,0.4,0.75,0.75,0.8,0.8,0.2,0.8,0.6,0.8],[0,0,0.6,0.8,0.8,0.6,0.75,0.4,0.4,0.8,0.8,0.2,0.8,0.6,0.8],[0,0.6,0,0.8,0.8,0.6,0.4,0.4,0.75,0.8,0.8,0.5,0.8,0.6,0.8],[0,0.8,0.8,0,1,0.8,0.6,0.6,0.6,1,1,0.4,1,0.8,1],[0,0.8,0.8,1,0,0.8,0.6,0.6,0.6,1,1,0.4,1,0.8,1],[0,0.6,0.6,0.8,0.8,0,0.4,0.4,0.4,0.8,0.8,0.5,0.8,0.6,0.8],[0,0.75,0.4,0.6,0.6,0.4,0,0.5,0.2,0.6,0.6,0.25,0.6,0.75,0.6],[0,0.4,0.4,0.6,0.6,0.4,0.5,0,0.5,0.6,0.6,0.25,0.6,0.75,0.6],[0,0.4,0.75,0.6,0.6,0.4,0.2,0.5,0,0.6,0.6,0.25,0.6,0.4,0.6],[0,0.8,0.8,1,1,0.8,0.6,0.6,0.6,0,1,0.4,1,0.8,1],[0,0.8,0.8,1,1,0.8,0.6,0.6,0.6,1,0,0.4,1,0.8,1],[0,0.2,0.5,0.4,0.4,0.5,0.25,0.25,0.25,0.4,0.4,0,0.4,0.5,0.4],[0,0.8,0.8,1,1,0.8,0.6,0.6,0.6,1,1,0.4,0,0.8,1],[0,0.6,0.6,0.8,0.8,0.6,0.75,0.75,0.4,0.8,0.8,0.5,0.8,0,0.8],[0,0.8,0.8,1,1,0.8,0.6,0.6,0.6,1,1,0.4,1,0.8,0]]
    return data

# def create_data_model():
#     """Stores the data for the problem."""
#     data = {}
#     data['Vs'] = ['1', '2']
#     data['Vc'] = ['3', '4', '5', '6', '7', '8', '9']
#     data['Q'] = [7, 5]
#     data['T'] = [12, 12]
#     data['U'] = [20, 20]
#     data['Ls'] = [12, 12]
#     data['Lc'] = [12, 12, 12, 12, 12, 12, 100]
#     data['Ss'] = [0, 0]
#     data['Sc'] = [0, 0, 0, 0, 0, 0, 0]
#     data['distance'] = [
#                         [10, 5, 7, 10, 1, 3, 10, 100],
#                         [10, 7, 9, 4, 11, 11, 12, 100],
#                         [5, 0, 2, 5, 4, 2, 5, 100],
#                         [3, 2, 0, 5, 6, 4, 3, 100],
#                         [6, 5, 5, 0, 9, 7, 8, 100],
#                         [9, 4, 6, 9, 0, 2, 9, 100],
#                         [7, 2, 4, 7, 2, 0, 7, 100],
#                         [1, 5, 3, 8, 9, 7, 0, 100],
#                         [100, 100, 100, 100, 100, 100, 100, 0],
#                         ]
#     data['duration'] = [
#                         [10, 5, 7, 10, 1, 3, 10, 100],
#                         [10, 7, 9, 4, 11, 11, 12, 100],
#                         [5, 0, 2, 5, 4, 2, 5, 100],
#                         [3, 2, 0, 5, 6, 4, 3, 100],
#                         [6, 5, 5, 0, 9, 7, 8, 100],
#                         [9, 4, 6, 9, 0, 2, 9, 100],
#                         [7, 2, 4, 7, 2, 0, 7, 100],
#                         [1, 5, 3, 8, 9, 7, 0, 100],
#                         [100, 100, 100, 100, 100, 100, 100, 0],
#                         ]
#     data['preference'] = [
#                         [1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1],
#                         [1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1],
#                         [1, 0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
#                         [1, 0.5, 0, 0.5, 0.5, 0.5, 0.5, 1],
#                         [1, 0.5, 0.5, 0, 0.5, 0.5, 0.5, 1],
#                         [1, 0.5, 0.5, 0.5, 0, 0.5, 0.5, 1],
#                         [1, 0.5, 0.5, 0.5, 0.5, 0, 0.5, 1],
#                         [1, 0.5, 0.5, 0.5, 0.5, 0.5, 0, 1],
#                         [1, 1, 1, 1, 1, 1, 1, 0],
#                         ]
#     data['jaccard'] = [
#                         [1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1],
#                         [1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1],
#                         [1, 0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
#                         [1, 0.5, 0, 0.5, 0.5, 0.5, 0.5, 1],
#                         [1, 0.5, 0.5, 0, 0.5, 0.5, 0.5, 1],
#                         [1, 0.5, 0.5, 0.5, 0, 0.5, 0.5, 1],
#                         [1, 0.5, 0.5, 0.5, 0.5, 0, 0.5, 1],
#                         [1, 0.5, 0.5, 0.5, 0.5, 0.5, 0, 1],
#                         [1, 1, 1, 1, 1, 1, 1, 0],
#                         ]
#     return data


# マツダエース
# def create_data_model():
#     """Stores the data for the problem."""
#     data = {}
#     data['Vs'] = ['4','5','6','8','9']
#     data['Vc'] = ['1','2','3','7','10']
#     data['Q'] = [2,2,2,2,2]
#     data['T'] = [3600,3600,3600,3600,3600]
#     data['Ls'] = [7200,3600,4800,4800,9000]
#     data['Lc'] = [4800,3600,7200,7200,9000]
#     data['Ss'] = [3600,0,0,0,3600]
#     data['Sc'] = [0,0,3600,3600,3600]
#     data['distance'] = [[24060,16740,28647,0,21278,19947],[11873,17726,6863,34042,13164,16939],[3472,7302,7144,23618,2739,6515],[4052,7644,7486,23960,3081,4990],[13182,9893,17769,26208,10400,6721],[7519,0,12106,16673,4737,3406],[6638,12106,0,28807,7928,11704],[24060,16673,28807,0,21278,19947],[3340,4737,7928,21278,0,4132],[6741,3406,11704,19947,4132,0]]
#     data['duration'] = [[2564,1882,3288,0,2288,2346],[1497,1997,863,3640,1557,2003],[456,829,1144,2472,389,835],[739,990,1305,2633,550,948],[1216,1101,1940,2744,940,594],[775,0,1499,1750,499,557],[1036,1499,0,3180,1096,1543],[2564,1750,3180,0,2288,2346],[366,499,1096,2288,0,511],[706,557,1543,2346,511,0]]
#     data['risk'] = [[1.1590909090909092,1,1.0681818181818181,1,1.0454545454545454,1],[1.1590909090909092,0,1.1590909090909092,1,1,1],[1.0568181818181819,1.1590909090909092,0,1.0681818181818181,1.1818181818181819,1.0681818181818181],[1.1590909090909092,1,1.0681818181818181,0,1.0454545454545454,1],[1.1590909090909092,1,1.1818181818181819,1.0454545454545454,0,1.0681818181818181],[1.1590909090909092,1,1.0681818181818181,1,1.0681818181818181,0],[1.0568181818181819,1.0568181818181819,1.0568181818181819,1.0568181818181819,1.1818181818181819,1.0568181818181819],[1.1590909090909092,0,1.1590909090909092,1,1,1],[1.0568181818181819,1.1590909090909092,0,1.0681818181818181,1.1818181818181819,1.0681818181818181],[1.1590909090909092,1,1.0681818181818181,0,1.0454545454545454,1],[1.1590909090909092,1,1.1818181818181819,1.0454545454545454,0,1.0681818181818181],[1.1590909090909092,1,1.0681818181818181,1,1.0681818181818181,0],[0.6818181818181819,0.625,0.6931818181818181,0.625,0.7159090909090909,0.6931818181818181],[0.7840909090909091,0,0.7840909090909091,0.625,0.625,0.625],[0.6818181818181819,0.7840909090909091,0,0.6931818181818181,0.8068181818181819,0.6931818181818181],[0.7840909090909091,0.625,0.6931818181818181,0,0.6704545454545454,0.625],[0.7840909090909091,0.625,0.8068181818181819,0.6704545454545454,0,0.6931818181818181],[0.7840909090909091,0.625,0.6931818181818181,0.625,0.6931818181818181,0],[1.4431818181818181,1.4204545454545454,1.6477272727272727,1.375,1.375,1.375],[1.5340909090909092,0,1.5340909090909092,1.375,1.375,1.375],[1.4318181818181819,1.5340909090909092,0,1.4431818181818181,1.5568181818181819,1.4431818181818181],[1.5340909090909092,1.375,1.4431818181818181,0,1.4204545454545454,1.375],[1.5340909090909092,1.375,1.5568181818181819,1.4204545454545454,0,1.4431818181818181],[1.5340909090909092,1.375,1.4431818181818181,1.375,1.4431818181818181,0],[1.875,1.4318181818181819,1.5795454545454546,1.5227272727272727,1.4659090909090908,1.4318181818181819],[1.5340909090909092,0,1.5340909090909092,1.375,1.375,1.375],[1.4318181818181819,1.5340909090909092,0,1.4431818181818181,1.5568181818181819,1.4431818181818181],[1.5340909090909092,1.375,1.4431818181818181,0,1.4204545454545454,1.375],[1.5340909090909092,1.375,1.5568181818181819,1.4204545454545454,0,1.4431818181818181],[1.5340909090909092,1.375,1.4431818181818181,1.375,1.4431818181818181,0]]
#     data['accidentCount'] = [[14,0,6,0,4,0],[14,0,14,0,0,0],[5,14,0,6,16,6],[14,0,6,0,4,0],[14,0,16,4,0,6],[14,0,6,0,6,0],[5,5,5,5,16,5],[14,0,14,0,0,0],[5,14,0,6,16,6],[14,0,6,0,4,0],[14,0,16,4,0,6],[14,0,6,0,6,0],[5,0,6,0,8,6],[14,0,14,0,0,0],[5,14,0,6,16,6],[14,0,6,0,4,0],[14,0,16,4,0,6],[14,0,6,0,6,0],[6,4,24,0,0,0],[14,0,14,0,0,0],[5,14,0,6,16,6],[14,0,6,0,4,0],[14,0,16,4,0,6],[14,0,6,0,6,0],[44,5,18,13,8,5],[14,0,14,0,0,0],[5,14,0,6,16,6],[14,0,6,0,4,0],[14,0,16,4,0,6],[14,0,6,0,6,0]]
#     return data


# マツダロジスティクス
# def create_data_model():
#     """Stores the data for the problem."""
#     data = {}
#     # data['Vs'] = ['4','5','6','8','10']
#     # data['Vc'] = ['1','2','3','7','9']
#     data['Vs'] = ['ye0NQtrfSUNv3gJnzQaG4nhfnkt1','chtkQtSVlYVDmXi8XyPfiJsMy0X2','YB7KSV2jrxegGz9lSHWhFVe5TzU2','8tDu6ZY5CCPF8oF0280s1oHsz7X2','VUWCbWJBDqMHUwYkyFXehwHs20K2']
#     data['Vc'] = ['9n3uDFx7t8PPQFmoKDZjwCnQYNn1','AzCbqU6wNgW8bPbk0cAYOmoiVrF2','nodSOz5RSYfYKRENMepo8r4kS8X2','jni3XymtlZOCwgbCU6w8K509jdv1','iL3LwDAW4BRM8am9xvKpYtxGv682']
#     data['Q'] = [2,2,2,2,2]
#     data['T'] = [3600,3600,3600,3600,3600]
#     data['Ls'] = [7200,4200,5400,5400,2700]
#     data['Lc'] = [6300,7200,2700,5400,3600]
#     data['Ss'] = [3600,0,0,0,0]
#     data['Sc'] = [0,3600,0,0,0]
#     data['distance'] = [[12225,10605,9900,9378,24533,18791],[1934,313,787,1618,16433,10691],[31817,32918,33042,33774,17940,22300],[15337,16438,16562,17294,0,5820],[4515,2895,2190,1781,17006,11264],[1776,0,1100,1931,16853,11111],[2744,1100,0,1409,16604,10862],[3614,1931,1409,0,17184,11442],[15337,16853,16604,17184,0,5820],[9564,11111,10862,11442,5820,0]]
#     data['duration'] = [[1941,1784,1706,1651,3056,2536],[223,66,221,352,1739,1219],[3032,3206,3260,3338,1689,2007],[1631,1805,1859,1937,0,606],[685,528,457,408,1902,1382],[225,0,312,443,1702,1182],[388,312,0,331,1783,1263],[571,443,331,0,1929,1409],[1631,1702,1783,1929,0,606],[1031,1182,1263,1409,606,0]]
#     data['risk'] = [[1.0454545454545454,1.0909090909090908,1.2272727272727273,1.1477272727272727,1.0454545454545454,1.0795454545454546],[1.1590909090909092,0,1.0454545454545454,1,1.0681818181818181,1.0568181818181819],[1.1022727272727273,1.0454545454545454,0,1.0454545454545454,1.0454545454545454,1.0568181818181819],[1.0795454545454546,1,1.0454545454545454,0,1,1.0568181818181819],[1.0568181818181819,1.0681818181818181,1.0454545454545454,1,0,1],[1.0568181818181819,1.0568181818181819,1.0568181818181819,1.0568181818181819,1,0],[1.5227272727272727,1.375,1.4204545454545454,1.4204545454545454,1.4318181818181819,1.5340909090909092],[1.5340909090909092,0,1.4204545454545454,1.375,1.4431818181818181,1.4318181818181819],[1.4772727272727273,1.4204545454545454,0,1.4204545454545454,1.4204545454545454,1.4318181818181819],[1.4545454545454546,1.375,1.4204545454545454,0,1.375,1.4318181818181819],[1.4318181818181819,1.4431818181818181,1.4204545454545454,1.375,0,1.375],[1.4318181818181819,1.4318181818181819,1.4318181818181819,1.4318181818181819,1.375,0],[1.4318181818181819,1.4318181818181819,1.5340909090909092,1.875,1.4545454545454546,1.375],[1.5340909090909092,0,1.4204545454545454,1.375,1.4431818181818181,1.4318181818181819],[1.4772727272727273,1.4204545454545454,0,1.4204545454545454,1.4204545454545454,1.4318181818181819],[1.4545454545454546,1.375,1.4204545454545454,0,1.375,1.4318181818181819],[1.4318181818181819,1.4431818181818181,1.4204545454545454,1.375,0,1.375],[1.4318181818181819,1.4318181818181819,1.4318181818181819,1.4318181818181819,1.375,0],[1.4318181818181819,1.4545454545454546,1.5227272727272727,1.875,1.375,1.375],[1.5340909090909092,0,1.4204545454545454,1.375,1.4431818181818181,1.4318181818181819],[1.4772727272727273,1.4204545454545454,0,1.4204545454545454,1.4204545454545454,1.4318181818181819],[1.4545454545454546,1.375,1.4204545454545454,0,1.375,1.4318181818181819],[1.4318181818181819,1.4431818181818181,1.4204545454545454,1.375,0,1.375],[1.4318181818181819,1.4318181818181819,1.4318181818181819,1.4318181818181819,1.375,0],[1.4545454545454546,1.375,1.5227272727272727,1.4318181818181819,1.4545454545454546,1.4318181818181819],[1.5340909090909092,0,1.4204545454545454,1.375,1.4431818181818181,1.4318181818181819],[1.4772727272727273,1.4204545454545454,0,1.4204545454545454,1.4204545454545454,1.4318181818181819],[1.4545454545454546,1.375,1.4204545454545454,0,1.375,1.4318181818181819],[1.4318181818181819,1.4431818181818181,1.4204545454545454,1.375,0,1.375],[1.4318181818181819,1.4318181818181819,1.4318181818181819,1.4318181818181819,1.375,0]]
#     data['accidentCount'] = [[4,8,20,13,4,7],[14,0,4,0,6,5],[9,4,0,4,4,5],[7,0,4,0,0,5],[5,6,4,0,0,0],[5,5,5,5,0,0],[13,0,4,4,5,14],[14,0,4,0,6,5],[9,4,0,4,4,5],[7,0,4,0,0,5],[5,6,4,0,0,0],[5,5,5,5,0,0],[5,5,14,44,7,0],[14,0,4,0,6,5],[9,4,0,4,4,5],[7,0,4,0,0,5],[5,6,4,0,0,0],[5,5,5,5,0,0],[5,7,13,44,0,0],[14,0,4,0,6,5],[9,4,0,4,4,5],[7,0,4,0,0,5],[5,6,4,0,0,0],[5,5,5,5,0,0],[7,0,13,5,7,5],[14,0,4,0,6,5],[9,4,0,4,4,5],[7,0,4,0,0,5],[5,6,4,0,0,0],[5,5,5,5,0,0]]
#     return data